In [2]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine

In [3]:
con_url = sa.engine.URL.create(
    drivername='mysql+pymysql',
    username='username',
    password='password',
    host='localhost',
    database='salesdb'
)

engine = create_engine(con_url)

In [4]:
query = '''
    select * from sales_table limit 10
'''

df = pd.read_sql_query(query, engine)
df.head()

,Order Number,Line Item,Order Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code,Gender,Name,...,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category,Exchange
0,366000,1,1/1/2016,265598,10,1304,1,CAD,Male,Tyler Vaught,...,Contoso Lens Adapter M450 White,Contoso,White,$31.27,$68.00,406,Cameras & Camcorders Accessories,4,Cameras and camcorders,1.3884
1,366001,1,1/1/2016,1269051,0,1048,2,USD,Male,Frank Upchurch,...,A. Datum SLR Camera X136 Silver,A. Datum,Silver,$141.47,$427.00,402,Digital SLR Cameras,4,Cameras and camcorders,1.0000
2,366001,2,1/1/2016,1269051,0,2007,1,USD,Male,Frank Upchurch,...,Fabrikam Microwave 1.5CuFt X1100 Black,Fabrikam,Black,$220.64,$665.94,803,Microwaves,8,Home Appliances,1.0000
3,366002,1,1/1/2016,266019,0,1106,7,CAD,Female,Joan Roche,...,Contoso SLR Camera M146 Orange,Contoso,Orange,$148.08,$322.00,402,Digital SLR Cameras,4,Cameras and camcorders,1.3884
4,366002,2,1/1/2016,266019,0,373,1,CAD,Female,Joan Roche,...,Adventure Works Laptop8.9 E0890 White,Adventure Works,White,$166.20,$326.00,301,Laptops,3,Computers,1.3884


In [5]:
from sqlalchemy import text

### 1. query to view total sales every year

In [6]:
query = text('''
    SELECT YEAR(STR_TO_DATE(`Order Date`, '%m/%d/%Y')) as year, SUM(Quantity) as total_Sales
    FROM sales_table
    GROUP BY year
''')


df = pd.read_sql_query(query, engine)
df

,year,total_Sales
0,2016,21725.0
1,2017,24784.0
2,2018,44498.0
3,2019,68404.0
4,2020,34463.0
5,2021,3797.0


### 2. query to view all the monthly sales in the year 2019


In [7]:
query = text('''
    SELECT YEAR(STR_TO_DATE(`Order Date`, '%m/%d/%Y')) as year,
           MONTH(STR_TO_DATE(`Order Date`, '%m/%d/%Y')) as months,
           SUM(Quantity) as monthly_Sales
    FROM sales_table
    WHERE YEAR(STR_TO_DATE(`Order Date`, '%m/%d/%Y')) = 2019
    GROUP BY months, year;
''')


df = pd.read_sql_query(query, engine)
df

,year,months,monthly_Sales
0,2019,1,6845.0
1,2019,2,7371.0
2,2019,3,3095.0
3,2019,4,562.0
4,2019,5,5960.0
5,2019,6,5497.0
6,2019,7,5328.0
7,2019,8,5922.0
8,2019,9,6033.0
9,2019,10,6403.0


### 3. query to see the maximum sale among all the years

In [8]:
query = text('''
    SELECT YEAR(STR_TO_DATE(`Order Date`, '%m/%d/%Y')) as year, SUM(Quantity) as max_Sale
    FROM sales_table
    GROUP BY year 
    ORDER BY max_Sale desc
    limit 1
''')


df = pd.read_sql_query(query, engine)
df

,year,max_Sale
0,2019,68404.0


### 4. query to see the minimum sale among all the years

In [9]:
query = text('''
    SELECT YEAR(STR_TO_DATE(`Order Date`, '%m/%d/%Y')) as year, SUM(Quantity) as min_Sale
    FROM sales_table
    GROUP BY year 
    ORDER BY min_Sale asc
    limit 1
''')


df = pd.read_sql_query(query, engine)
df

,year,min_Sale
0,2021,3797.0


### 5. view total sales for each product category

In [10]:
query = text('''
    SELECT Category, SUM(Quantity) as total_Sales
    FROM sales_table
    GROUP BY Category
    ORDER BY total_Sales DESC
    LIMIT 5
''')

df = pd.read_sql_query(query, engine)
df

,Category,total_Sales
0,Computers,44140.0
1,Cell phones,31452.0
2,"Music, Movies and Audio Books",28796.0
3,Audio,23483.0
4,Games and Toys,22591.0


### 6. Query for total sales based on product subcategory

In [11]:
query = text('''
    SELECT Subcategory, SUM(Quantity) as total_Sales
    FROM sales_table
    GROUP BY Subcategory
    ORDER BY total_Sales DESC
    LIMIT 5
''')

df = pd.read_sql_query(query, engine)
df

,Subcategory,total_Sales
0,Movie DVD,28796.0
1,Desktops,20620.0
2,Bluetooth Headphones,13649.0
3,Download Games,11547.0
4,Boxed Games,11044.0


### 7. Query for top 5 countries with high Average sales 

In [12]:
query = text('''
    SELECT Country_y as Country, AVG(Quantity) as average_Sales
    FROM sales_table
    GROUP BY Country
    ORDER BY average_Sales DESC
    LIMIT 5
''')

df = pd.read_sql_query(query, engine)
df

,Country,average_Sales
0,Netherlands,3.2521
1,Germany,3.1918
2,Italy,3.1781
3,Australia,3.1517
4,United States,3.1496


### 8. query to see top 2 countries with maximum sale quantity and their average store size in sq meters

In [13]:
query = text('''
    select Country_y, AVG(`Square Meters`) as avg_sq_meters, SUM(Quantity) as Max_Sale
    from sales_table
    group by Country_y 
    order by Max_Sale desc
    limit 2
''')


df = pd.read_sql_query(query, engine)
df

,Country_y,avg_sq_meters,Max_Sale
0,United States,1637.421013,83638.0
1,Online,0.000000,41305.0


### 9. query for top 5 Store sizes with maximum Sale quantity

In [14]:
query = text('''
    select `Square Meters`, sum(Quantity) as Total_Sales
    from sales_table
    group by `Square Meters`
    order by Total_Sales desc
    limit 5
''')


df = pd.read_sql_query(query, engine)
df

,Square Meters,Total_Sales
0,2000.0,50152.0
1,0.0,41305.0
2,2100.0,10856.0
3,1260.0,8208.0
4,1120.0,6613.0


### 10. top 5 high sale quantity by thier store sizes and their respective country 

In [15]:
query = text('''
    select Country_y, `Square Meters`, sum(Quantity) as Total_Sales
    from sales_table
    group by `Square Meters`, Country_y
    order by Total_Sales desc
    limit 5
''')


df = pd.read_sql_query(query, engine)
df

,Country_y,Square Meters,Total_Sales
0,Online,0.0,41305.0
1,United States,2000.0,39263.0
2,United States,1260.0,8208.0
3,United Kingdom,2100.0,7691.0
4,United States,1120.0,6613.0


### 11.top 10 Most sold product in 'Movie DVD' subcategory and by Category

In [19]:
query = text('''
    select Category, Subcategory, `Product Name`, sum(Quantity) as tot_sales
    from sales_table
    where Subcategory = 'Movie DVD'
    group by `Product Name`,Subcategory, Category
    order by tot_sales desc
    limit 10
''')


df = pd.read_sql_query(query, engine)
df

,Category,Subcategory,Product Name,tot_sales
0,"Music, Movies and Audio Books",Movie DVD,SV DVD Player M120 White,404.0
1,"Music, Movies and Audio Books",Movie DVD,SV DVD 12-Inch Player Portable M400 White,401.0
2,"Music, Movies and Audio Books",Movie DVD,SV DVD 14-Inch Player Portable L100 Silver,391.0
3,"Music, Movies and Audio Books",Movie DVD,SV DVD External DVD Burner M200 Black,384.0
4,"Music, Movies and Audio Books",Movie DVD,Contoso DVD 55DVD Storage Binder M56 Silver,380.0
5,"Music, Movies and Audio Books",Movie DVD,Contoso DVD 7-Inch Player Portable E200 Black,376.0
6,"Music, Movies and Audio Books",Movie DVD,Contoso DVD Player M100 Black,376.0
7,"Music, Movies and Audio Books",Movie DVD,SV DVD Recorder L210 Silver,372.0
8,"Music, Movies and Audio Books",Movie DVD,Contoso DVD Recorder L230 Grey,371.0
9,"Music, Movies and Audio Books",Movie DVD,Contoso DVD 7-Inch Player Portable E200 Silver,370.0


### 12. top 10 Most sold product in 'Desktop' subcategory and by Category

In [20]:
query = text('''
    select Category, Subcategory, `Product Name`, sum(Quantity) as tot_sales
    from sales_table
    where Subcategory = 'Desktops'
    group by `Product Name`,Subcategory, Category 
    order by tot_sales desc
    limit 10
''')


df = pd.read_sql_query(query, engine)
df

,Category,Subcategory,Product Name,tot_sales
0,Computers,Desktops,WWI Desktop PC2.33 X2330 Black,550.0
1,Computers,Desktops,WWI Desktop PC1.80 E1800 White,538.0
2,Computers,Desktops,Adventure Works Desktop PC2.30 MD230 White,521.0
3,Computers,Desktops,Adventure Works Desktop PC1.60 ED160 Black,521.0
4,Computers,Desktops,Adventure Works Desktop PC1.80 ED180 Black,520.0
5,Computers,Desktops,Adventure Works Desktop PC2.30 MD230 Black,514.0
6,Computers,Desktops,WWI Desktop PC1.60 E1600 Black,509.0
7,Computers,Desktops,WWI Desktop PC1.60 E1600 Silver,507.0
8,Computers,Desktops,WWI Desktop PC1.80 E1801 Black,505.0
9,Computers,Desktops,Adventure Works Desktop PC1.60 ED160 White,505.0


### 12. top 10 Most sold product in 'Contoso' Brand and by Category and subcategory

In [22]:
query = text('''
    select Brand, Category, Subcategory, `Product Name`, sum(Quantity) as tot_sales
    from sales_table
    where Brand = 'Contoso'
    group by `Product Name`,Subcategory, Category, Brand
    order by tot_sales desc
    limit 10
''')


df = pd.read_sql_query(query, engine)
df

,Brand,Category,Subcategory,Product Name,tot_sales
0,Contoso,"Music, Movies and Audio Books",Movie DVD,Contoso DVD 55DVD Storage Binder M56 Silver,380.0
1,Contoso,"Music, Movies and Audio Books",Movie DVD,Contoso DVD Player M100 Black,376.0
2,Contoso,"Music, Movies and Audio Books",Movie DVD,Contoso DVD 7-Inch Player Portable E200 Black,376.0
3,Contoso,"Music, Movies and Audio Books",Movie DVD,Contoso DVD Recorder L230 Grey,371.0
4,Contoso,"Music, Movies and Audio Books",Movie DVD,Contoso DVD 7-Inch Player Portable E200 Silver,370.0
5,Contoso,"Music, Movies and Audio Books",Movie DVD,Contoso DVD 12-Inch Player Portable M400 Silver,370.0
6,Contoso,"Music, Movies and Audio Books",Movie DVD,Contoso DVD Player M110 Silver,360.0
7,Contoso,"Music, Movies and Audio Books",Movie DVD,Contoso DVD External DVD Burner M200 Grey,360.0
8,Contoso,"Music, Movies and Audio Books",Movie DVD,Contoso DVD 55DVD Storage Binder M56 Red,357.0
9,Contoso,"Music, Movies and Audio Books",Movie DVD,Contoso DVD 38 DVD Storage Binder E25 Black,356.0
